<a href="https://colab.research.google.com/github/kmk4444/Autonomic_agents/blob/main/Part2_advanced_react.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will develop advanced react application.
There are three language models.
There are two search models.
There are two creating image models.

**Requirements.txt**

In [4]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchainhub >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo streamlit >> requirements.txt
!echo duckduckgo-search >> requirements.txt
!echo anthropic >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo autogenstudio >> requirements.txt
!echo crewai >> requirements.txt

**terminal / bash komutu**

In [ ]:
pip install -r requirements.txt

In [6]:
#load_tools is duckduckgo search method.
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatAnthropic
from langchain import hub
from langchain_community.callbacks import StreamlitCallbackHandler
from langchain_community.tools.tavily_search import TavilySearchResults
import streamlit as st
import os
import customtools
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
#my_key_anthropic = os.getenv("anthropic_apikey")
#os.environ["TAVILY_API_KEY"] = os.getenv("tavily_apikey")

my_key_openai="---"
my_key_google="---"
my_key_anthropic="---"
os.environ["TAVILY_API_KEY"]="----"

llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-pro")
llm_gpt = ChatOpenAI(api_key=my_key_openai, model="gpt-4-0125-preview", temperature=0, streaming=True)
llm_claude = ChatAnthropic(anthropic_api_key=my_key_anthropic, model_name="claude-2.1")

#System prompt, we call system prompt from langchain hub.
agent_prompt = hub.pull("hwchase17/react")





ModuleNotFoundError: No module named 'customtools'